In [1]:
from config.constants import MONGODB_CREDENTIALS, SQL_LEAGUES_CONN, SQL_LEAGUES_ENGINE, API_KEY
from pymongo import MongoClient
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from riotwatcher import RiotWatcher

# MongoDB

In [2]:
mongodb = MongoClient(MONGODB_CREDENTIALS).slds

In [3]:
soloq_m_coll = mongodb.soloq_m
soloq_tl_coll = mongodb.soloq_tl

In [28]:
slo_tl_coll = mongodb.slo_tl

In [ ]:
slo_tl_coll.find_one({'gameId':})

In [4]:
sql = pymysql.connect(**SQL_LEAGUES_CONN)

In [5]:
cursor1 = sql.cursor()
query = 'SELECT game_id, realm, hash FROM slo'
cursor1.execute(query)
new_game_ids = [(gid[0], gid[1], gid[2]) for gid in cursor1]
platforms = [gid[1] for gid in new_game_ids]

In [6]:
set(platforms)

{'ESPORTSTMNT01', 'ESPORTSTMNT05', 'EUW1'}

In [7]:
cursor = soloq_m_coll.find({'platformId': {'$in': list(set(platforms))}}, {'_id': 0, 'gameId': 1, 'platformId': 1})

In [8]:
current_game_ids = [(gid['gameId'], gid['platformId']) for gid in cursor]

In [12]:
df = pd.DataFrame(current_game_ids)

In [49]:
dict1

{'gameCreation': {'$gteeee': 'meirae'}}

In [31]:
gs = soloq_m_coll.find({'gameCreation': {'$gte': 1526335200000, '$lte': 1526507999000}}, {'_id': 0, 'gameId': 1})

In [32]:
gids = [obj['gameId'] for obj in gs]

In [33]:
len(gids)

356

### Regex in MongoDB with game versions

In [193]:
patch = '8.1'

In [194]:
rgex_patch = '^' + '{}'.format(patch).replace('.', r'\.')

In [195]:
gamevers = [game['gameVersion'] for game in soloq_m_coll.find({'platformId': region, 'gameVersion': {'$regex': rgex_patch}}, {'_id': 0, 'gameVersion': 1})]

In [196]:
pd.Series(gamevers).value_counts()

8.1.214.5847    14
8.1.213.4336    13
dtype: int64

In [ ]:
g = soloq_m_coll.find({'participantIdentities.player.accountId': {'$in': [23787352]}}, {'_id': 0, 'gameId': 1})

In [ ]:
gs = [obj['gameId'] for obj in g]

In [ ]:
pd.DataFrame(gs).rename(columns={0: 'game_id'})

# SQL

In [2]:
import pymysql

In [16]:
df = pd.read_csv('C:\\Users\\david\\Downloads\\slo.csv', encoding='ISO-8859-1')

In [17]:
cnx = create_engine(SQL_LEAGUES_ENGINE)

In [18]:
df.to_sql(con=cnx, name='slo', if_exists='replace')

In [82]:
teams = sql_df[['team_name', 'team_abbv']]

In [83]:
players = sql_df[['region', 'main_role', 'substitute', 'player_name', 'account_id', 'id', 'account_type']]

# Random

In [90]:
from riotwatcher import RiotWatcher
from config.constants import API_KEY
rw = RiotWatcher(API_KEY)

In [26]:
summ = rw.summoner.by_name(region='EUW1', summoner_name='Missing Q')

In [28]:
accid = summ['accountId']

In [30]:
ml = rw.match.matchlist_by_account(account_id=accid, region='EUW1')

In [35]:
gid = ml['matches'][0]['gameId']

In [36]:
gid

3669456176

In [44]:
tl = rw.match.timeline_by_match(match_id=3668417003, region='EUW1')

In [47]:
from itertools import chain

In [49]:
events = list(chain.from_iterable([frame['events'] for frame in tl['frames']]))

In [56]:
ward_df = pd.DataFrame([ward for ward in events if ward['type'] == 'WARD_PLACED'])

In [75]:
positions = {1: 'JUNG', 2: 'SUPP', 3: 'MID', 4: 'TOP', 5: 'ADC', 6: 'TOP', 7: 'JUNG', 8: 'MID', 9: 'ADC', 10:'SUPP'}

In [82]:
df_pos = pd.DataFrame(positions, index=(0,)).T.reset_index().rename(columns={'index': 'pid'})

In [85]:
ward_df2 = ward_df.merge(df_pos, left_on='creatorId', right_on='pid')

In [86]:
ward_df2.sort_values(['creatorId', 'timestamp']).to_excel('ward_test.xlsx')